In [ ]:
# ===============================
# Step 1: Mount Google Drive
# ===============================
from google.colab import drive
drive.mount('/content/drive')

# ===============================
# Step 2: Imports
# ===============================
import gradio as gr
import tensorflow as tf
import numpy as np
import xgboost as xgb
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
import joblib
from PIL import UnidentifiedImageError

# ===============================
# Step 3: Load feature extractor (same as training)
# ===============================
feature_extractor = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

# ===============================
# Step 4: Load trained XGBoost model
# ===============================
xgb_model = xgb.XGBClassifier()
xgb_model.load_model("/content/drive/MyDrive/xgboost_hybrid_model.json")

# ===============================
# Step 5: Define class names
# ===============================
class_names = ["lung_aca", "lung_n", "lung_scc"]

# ===============================
# Step 6: Preprocess + Predict function
# ===============================
def preprocess_and_predict(img):
    try:
        # Convert image for feature extractor
        img = img.resize((224, 224))  # match training input size
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # Extract CNN features
        features = feature_extractor.predict(img_array)

        # Predict with XGBoost
        preds = xgb_model.predict_proba(features)[0]

        # Get top prediction and its confidence
        top_pred_index = np.argmax(preds)
        top_pred_class = class_names[top_pred_index]
        top_confidence = float(preds[top_pred_index])

        # Check if confidence is less than 70% (0.7)
        if top_confidence < 0.7:
            # Return all class probabilities if confidence is low
            return dict(zip(class_names, map(float, preds)))
        else:
            # Otherwise, return only the top prediction and its confidence
            return {top_pred_class: top_confidence}
    except UnidentifiedImageError:
        return "Please upload a proper image."
    except Exception as e:
        return f"An error occurred: {e}"


# ===============================
# Step 7: Gradio Interface
# ===============================
demo = gr.Interface(
    fn=preprocess_and_predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="Lung Cancer Subtype Classifier",
    description="Upload a histopathological image to classify into: lung_aca, lung_n, or lung_scc.",
    flagging_mode='never' # Remove the flagging feature
)

demo.launch(share=True)

Mounted at /content/drive


/tmp/ipython-input-2137488993.py:23: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  feature_extractor = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d51d8ee423a3294d3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
